In [94]:
import json
from Bio import Entrez, SeqIO, AlignIO
from sklearn.neighbors import NearestNeighbors
from Bio.Phylo.TreeConstruction import DistanceCalculator
import heapq
import numpy as np
import pandas as pd
import json
import subprocess
import random

In [5]:
Entrez.email = 'alekey039@hotmail.com'

In [35]:
with open('receptor_data.json', 'r') as file: #use read_data() function
        receptor_data = json.load(file)

In [36]:
with open('phagedicts.json', 'r') as f: #use read_data() function
    phageinfo = json.load(f)

In [37]:
labels = sorted(set(receptor_data.keys())) 
proteins = set()
for ls in receptor_data.values():
    proteins.update(ls)
proteins = sorted(proteins)

df = pd.DataFrame(index = labels, columns = proteins)

In [38]:
for sp in df.index:
    df.loc[sp] = df.columns.isin(receptor_data.get(sp, False)).astype(int)

In [31]:
# Check membership in current dataframe with pathogenic species info

def membership(target, df):
    if target in df.index:
        return True
    else:
        return False
        
def produce_array(target, df):
    if membership(target, df):
        return None
    else:
        targetdict = {}
        targetdict[target] = receptors(target)
        target_conf = df.columns.isin(targetdict.get('E. coli', False)).astype(int)
        target_features = target_conf.reshape(1, -1)
        return target_features 

In [32]:
def remove_ifmember(target_features, target, df): # receive here the df and array for kNN

    if target_features is None:
        target_row = df.loc[target]
        target_features = target_row.values.reshape(1, -1)
        features_data = df.drop(target, axis = 0).values

    else:
        pass
        
    return target_features, features_data
    

In [59]:
def nearest_bacteria(target_features, features_data, neighbors = 1):
    nn_model = NearestNeighbors(n_neighbors = neighbors, metric = 'hamming')
    nn_model.fit(features_data)

    distances, indices = nn_model.kneighbors(target_features)

    return distances, indices
    
def nearest_names(indices, df):
    similar = list(df.index[indices[0]].values)
    
    return similar

In [41]:
def nearest_phages(similar, phageinfo):
    similar_phages = {host:[record['acc'] for record in phageinfo if record['host'] == host] for host in similar}

    return similar_phages

In [42]:
# Retrieve genomes
# One neighbor at a time / multiples fasta files

def phage_genomes(similar_phages):
        
        filename_ls = []
        for bact, phages in similar_phages.items():
            if len(phages) > 2:
                filename = f'{"_".join(bact.split())}_phages.fasta'
                filename_ls.append(filename)
                seqs = []
                
                handle = Entrez.efetch(db='nucleotide', id = phages, rettype = 'fasta', retmode = 'text')
                for record in SeqIO.parse(handle, 'fasta'):
                    seqs.append(record)
                handle.close()
            
            with open(filename, 'w') as file: 
                SeqIO.write(seqs, file, 'fasta')
            
        return filename_ls

In [43]:
# Alignment function

def align_sequences(input_file, output_file):
    print(f'Running alignment: {input_file}')
    mafft_command = f'mafft --auto --quiet {input_file} > {output_file}'
    subprocess.run(mafft_command, shell=True)

In [44]:
def distance_matrix(input_file):
    calculator = DistanceCalculator('identity')
    alignment = AlignIO.read(input_file, 'fasta')
    matrix = calculator.get_distance(alignment)
    
    return matrix

In [45]:
def top_distances(matrix, k = 1):
    array = np.array(matrix)

    minheap = [] # Initialize heap structure

    for i in range(len(array)):
        for j in range(i):
            d = array[i,j] # Distance between two samples
            if len(minheap) < k: # If heap is less than k, add the element
                heapq.heappush(minheap, (d, i, j)) 
            elif d > minheap[0][0]: # If distance is greater than root
                # push new element and pop current
                heapq.heappushpop(minheap, (d, i, j))
    return minheap

In [2]:
def most_diverse_phages(filename_ls): #time consuming, genome alignment
    sorted_distances_list = []
    phage_matrix_list = []
    
    for file in filename_ls:
        aligned_file = f'aligned{file}'
        align_sequences(file, aligned_file)
        phage_matrix = distance_matrix(aligned_file)
        sorted_distances = top_distances(phage_matrix, k = 1)
        
        phage_matrix_list.append(phage_matrix)
        sorted_distances_list.append(sorted_distances)
        
    return sorted_distances_list, phage_matrix_list 

def indices_to_accn(sorted_distances_list, phage_matrix_list):
    diverse_accn = []
    for sorted_distances, phage_matrix in sorted_distances_list, phage_matrix_list:
        unique_indices = set()
        
        for _, i, j in sorted_distances: 
            unique_indices.update([i, j])
            
        diverse_accn.extend(phage_matrix.names[ind] for ind in unique_indices)
    return diverse_accn

In [103]:
def accession_cocktail(diverse_accn, similar_phages):
    candidate_accs = []
    accn_set = set(diverse_accn)
    
    for bact, accns in similar_phages.items(): 
        if len(accns) > 2:
            candidate_accs.extend([accn for accn in accns if accn in accn_set])
        else:
            candidate_accs.extend(accns)
    return candidate_accs

In [104]:
def final_cocktail(candidate_accs):
    
    return [rec['phage'] for rec in phageinfo if rec['acc'] in candidate_accs]

In [107]:
def alignment_choice():
    valid = False
    while not valid:
        choice = int(input('How would you like to select phages?\n'
                       '1. Choose phages randomly\n'
                       '2. Align phage genomes for maximum diversity\n'
                       'Enter your choice (1 or 2): '))
        if choice == 1 or choice == 2:
            valid = True
        else: 
            print('Invalid choice. Please try again.')
    return int(choice)

In [105]:
def random_cocktail(similar_phages):
    random_accs = []
    phagelist = similar_phages.values()
    minrand = len(min(phagelist,key = len))
    valid = False
    while not valid:
        try:
            selection_str = input(f'Choose up to {minrand} phages for each species\n')
            selection = int(selection_str)
            if selection <= minrand:
                valid = True
            else:
                print('Invalid choice. Please choose a valid number.')
        except ValueError:
            print('Invalid choice. Please enter an integer.')
            
    for phages in phagelist:
        random_phages = random.sample(phages, selection)
        random_accs.extend(random_phages)
        
    return random_accs

In [111]:
# Implementation of the functions
target = 'Escherichia coli'
target_features = produce_array(target, df)
target_features, features_data = remove_ifmember(target_features, target, df)
distances, indices = nearest_bacteria(target_features, features_data, neighbors = 3)
similar = nearest_names(indices, df)
similar_phages = nearest_phages(similar, phageinfo)

choice = alignment_choice()

if choice == 1:
    random_accs = random_cocktail(similar_phages)
    product = final_cocktail(random_accs)
    
else:
    filename_ls = phage_genomes(similar_phages)
    sorted_distances, phage_matrix = most_diverse_phages(filename_ls)
    diverse_accn = indices_to_accn(phage_distances, phage_matrix)
    candidate_accs = accession_cocktail(diverse_accn, similar_phages)
    product = final_cocktail(candidate_accs)

print(product)

How would you like to select phages?
1. Choose phages randomly
2. Align phage genomes for maximum diversity
Enter your choice (1 or 2):  1
Choose up to 1 phages for each species
 1


['Shewanella phage vB_SspM_M16-3', 'Escherichia phage EP75', 'Shigella phage SFN6B']
